# Genie Top200 Chart

In [2]:
import requests
import pandas as pd


## 1. 인터넷상에서 첫번째 데이터 가져오기

In [3]:
# Genie top 200
url = "https://www.genie.co.kr/chart/top200"
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [5]:
# 기계가 아니라는 알림
header = {"user-Agent":"Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36" }
req = requests.get(url, headers=header)
html = req.text
#html

In [7]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")

## 2. 찾으려고 하는 데이터의 태그 찾기

In [10]:
# <table class="list-wrap">
table = soup.select_one("table.list-wrap")
trs = table.select("tr.list")
len(trs)

50

In [11]:
trs = soup.select("tr.list")
len(trs)

50

## 3. 여러개의 데이터중 하나를 선택해서 원하는 정보 추출

In [12]:
# rank, title, artist, album
tr = trs[0]

In [14]:
#rank
tr.select_one(".number").get_text()

'1\n                                        \n                                    \n유지\n\n'

In [16]:
tr.select_one(".number").get_text().split("\n")[0]

'1'

In [21]:
rank=int(tr.select_one(".number").get_text().split("\n")[0])
rank

1

In [23]:
# title
tr.select_one(".info").find("a").get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                Queendom'

In [34]:
title = tr.select_one(".info").find("a").get_text().strip()
title

'Queendom'

In [28]:
# artist
tr.select_one(".info").select_one(".artist").get_text()

'Red Velvet (레드벨벳)'

In [31]:
artist = tr.select_one(".info").select_one(".artist").get_text().strip()
artist

'Red Velvet (레드벨벳)'

In [32]:
# album
tr.select_one(".info").select_one(".albumtitle").get_text().strip()

'Queendom - The 6th Mini Album'

In [33]:
album = tr.select_one(".info").select_one(".albumtitle").get_text().strip().strip()
album

'Queendom - The 6th Mini Album'

## 4. 한페이지에 있는 모든 데이터를 반복문으로 가져오기

In [36]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank=int(tr.select_one(".number").get_text().split("\n")[0])
    title = tr.select_one(".info").find("a").get_text().strip()
    artist = tr.select_one(".info").select_one(".artist").get_text().strip()
    album = tr.select_one(".info").select_one(".albumtitle").get_text().strip().strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)


In [42]:
df = pd.DataFrame({
        "순위" : rank_list
        , "곡명" : title_list
        , "가수" : artist_list
        , "앨범" : album_list
})
df.tail()

,순위,곡명,가수,앨범
45,46,사이렌 Remix (Feat. UNEDUCATED KID & Paul Blanco),호미들,사이렌 Remix
46,47,어푸 (Ah puh),아이유 (IU),IU 5th Album 'LILAC'
47,48,가을 우체국 앞에서,김대명,슬기로운 의사생활 시즌2 OST Part 2 (tvN 목요드라마)
48,49,Life Goes On,방탄소년단,BE
49,50,그대라는 사치,임영웅,사랑의 콜센타 PART62


In [43]:
lines = []
for tr in trs:
    rank = int(tr.select_one(".number").get_text().split("\n")[0])
    title = tr.select_one(".info").find("a").get_text().strip()
    artist = tr.select_one(".info").select_one(".artist").get_text().strip()
    album = tr.select_one(".info").select_one(".albumtitle").get_text().strip().strip()
    lines.append([rank, title, artist, album])

In [45]:
df = pd.DataFrame(lines, columns=["순위","곡명","가수","엘범"])
df.head()

,순위,곡명,가수,엘범
0,1,Queendom,Red Velvet (레드벨벳),Queendom - The 6th Mini Album
1,2,신호등,이무진,신호등
2,3,낙하 (With 아이유),AKMU (악뮤),NEXT EPISODE
3,4,바라만 본다,MSG워너비 (M.O.M),MSG워너비 1집
4,5,Weekend,태연 (TAEYEON),Weekend


### 5. 모든 페이지 데이터 가져오기

In [46]:
sub = "https://www.genie.co.kr/chart/top200?ditc=D&ymd=20210817&hh=14&rtm=Y&pg="
rank_list, title_list, artist_list, album_list = [],[],[],[]
for page in range(1,5):
    url = f"{sub}{page}"
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(html, "html.parser")
    trs = soup.select("tr.list")
    for tr in trs:
        rank=int(tr.select_one(".number").get_text().split("\n")[0])
        title = tr.select_one(".info").find("a").get_text().strip()
        artist = tr.select_one(".info").select_one(".artist").get_text().strip()
        album = tr.select_one(".info").select_one(".albumtitle").get_text().strip().strip()
        rank_list.append(rank)
        title_list.append(title)
        artist_list.append(artist)
        album_list.append(album)    

In [47]:
df = pd.DataFrame({
          "순위" : rank_list
        , "곡명" : title_list
        , "가수" : artist_list
        , "앨범" : album_list
})
df.tail()

,순위,곡명,가수,앨범
195,46,사이렌 Remix (Feat. UNEDUCATED KID & Paul Blanco),호미들,사이렌 Remix
196,47,어푸 (Ah puh),아이유 (IU),IU 5th Album 'LILAC'
197,48,가을 우체국 앞에서,김대명,슬기로운 의사생활 시즌2 OST Part 2 (tvN 목요드라마)
198,49,Life Goes On,방탄소년단,BE
199,50,그대라는 사치,임영웅,사랑의 콜센타 PART62


In [48]:
df.to_csv("GenieTop200(210817).csv",index=False,encoding='utf-8-sig')